In [18]:
import mlflow
from mlflow.tracking import MlflowClient
mlflow.set_tracking_uri("http://localhost:5000")
import rich
import pprint
client = MlflowClient()

## Python Dictionary

In [19]:
experiment_name = "1-REG"
exp = client.get_experiment_by_name(experiment_name)
exp

<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1764418165659, experiment_id='1', last_update_time=1764418165659, lifecycle_stage='active', name='1-REG', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [20]:
runs = client.search_runs(
        experiment_ids=[exp.experiment_id],
        filter_string="attributes.status = 'FINISHED'",
        order_by=["attributes.start_time DESC"],
        # max_results=1,
    )
print("Number of runs :",len(runs))
pprint.pprint(runs)

Number of runs : 16
[<Run: data=<RunData: metrics={'accuracy': 0.279920369386673,
 'loss': 1.374122142791748,
 'val_accuracy': 0.3031262457370758,
 'val_loss': 1.3692481517791748,
 'validation_accuracy': 0.3031262457370758,
 'validation_loss': 1.3692481517791748}, params={'batch_size': '400',
 'class_weight': 'None',
 'epochs': '1',
 'initial_epoch': '0',
 'opt_amsgrad': 'False',
 'opt_beta_1': '0.9',
 'opt_beta_2': '0.999',
 'opt_clipnorm': 'None',
 'opt_clipvalue': 'None',
 'opt_ema_momentum': '0.99',
 'opt_ema_overwrite_frequency': 'None',
 'opt_epsilon': '1e-07',
 'opt_global_clipnorm': 'None',
 'opt_gradient_accumulation_steps': 'None',
 'opt_learning_rate': '0.0010000000474974513',
 'opt_loss_scale_factor': 'None',
 'opt_name': 'adam',
 'opt_use_ema': 'False',
 'opt_weight_decay': 'None',
 'sample_weight': 'None',
 'shuffle': 'True',
 'steps_per_epoch': 'None',
 'validation_batch_size': 'None',
 'validation_freq': '1',
 'validation_split': '0.0',
 'validation_steps': 'None'}, tag

In [21]:
dict(runs[0]).keys()

dict_keys(['data', 'info', 'inputs', 'outputs'])

In [22]:
run = runs[0].to_dictionary()
run

{'info': {'artifact_uri': 'mlflow-artifacts:/1/c2a0423253964f48b4b5adc042eb5a4f/artifacts',
  'end_time': 1764510510772,
  'experiment_id': '1',
  'lifecycle_stage': 'active',
  'run_id': 'c2a0423253964f48b4b5adc042eb5a4f',
  'run_name': 'rnn_16',
  'start_time': 1764510438174,
  'status': 'FINISHED',
  'user_id': 'srirama'},
 'data': {'metrics': {'accuracy': 0.279920369386673,
   'loss': 1.374122142791748,
   'val_accuracy': 0.3031262457370758,
   'val_loss': 1.3692481517791748,
   'validation_accuracy': 0.3031262457370758,
   'validation_loss': 1.3692481517791748},
  'params': {'epochs': '1',
   'batch_size': '400',
   'validation_split': '0.0',
   'shuffle': 'True',
   'class_weight': 'None',
   'sample_weight': 'None',
   'initial_epoch': '0',
   'steps_per_epoch': 'None',
   'validation_steps': 'None',
   'validation_batch_size': 'None',
   'validation_freq': '1',
   'opt_name': 'adam',
   'opt_learning_rate': '0.0010000000474974513',
   'opt_weight_decay': 'None',
   'opt_clipnor

In [23]:
from pygments import highlight
from pygments.lexers import YamlLexer
from pygments.formatters import Terminal256Formatter
import yaml

yaml_str = yaml.dump(run, sort_keys=False, default_flow_style=False)

# Strong bright colors for dark background
formatter = Terminal256Formatter(style="monokai")  
print(highlight(yaml_str, YamlLexer(), formatter))


info:
  artifact_uri: mlflow-artifacts:/1/c2a0423253964f48b4b5adc042eb5a4f/artifacts
  end_time: 1764510510772
  experiment_id: '1'
  lifecycle_stage: active
  run_id: c2a0423253964f48b4b5adc042eb5a4f
  run_name: rnn_16
  start_time: 1764510438174
  status: FINISHED
  user_id: srirama
data:
  metrics:
    accuracy: 0.279920369386673
    loss: 1.374122142791748
    val_accuracy: 0.3031262457370758
    val_loss: 1.3692481517791748
    validation_accuracy: 0.3031262457370758
    validation_loss: 1.3692481517791748
  params:
    epochs: '1'
    batch_size: '400'
    validation_split: '0.0'
    shuffle: 'True'
    class_weight: None
    sample_weight: None
    initial_epoch: '0'
    steps_per_epoch: None
    validation_steps: None
    validation_batch_size: None
    validation_freq: '1'
    opt_name: adam
    opt_learning_rate: '0.0010000000474974513'
    opt_weight_decay: None
    opt_clipnorm: None
    opt_global_clipnorm: None
    opt_clipvalue: None
    opt_use_ema: 'False'
    opt_ema_

## Model Promotion

In [24]:
TRACKING_URI = "http://localhost:5000"
EXPERIMENT_NAME = "1-REG"
mlflow.set_tracking_uri(TRACKING_URI)

client = MlflowClient(TRACKING_URI)
experiment = client.get_experiment_by_name(EXPERIMENT_NAME)
if not experiment:
    raise ValueError(f"Experiment '{EXPERIMENT_NAME}' not found.")
# Fetch latest FINISHED run
runs = client.search_runs(
    experiment_ids=[experiment.experiment_id],
    filter_string="attributes.status = 'FINISHED'",
    order_by=["attributes.start_time DESC"],
    max_results=1,
)
if not runs:
    raise ValueError("No finished runs found for this experiment.")
latest_run = runs[0] # Traet this like dot access dicitonary
latest_run_id = latest_run.info.run_id
run_name = latest_run.info.run_name
print(f"Latest runId : {latest_run_id} and name : {run_name}")

Latest runId : c2a0423253964f48b4b5adc042eb5a4f and name : rnn_16


In [39]:
experiment.__dict__

{'_experiment_id': '1',
 '_name': '1-REG',
 '_artifact_location': 'mlflow-artifacts:/1',
 '_lifecycle_stage': 'active',
 '_tags': {'mlflow.experimentKind': 'custom_model_development'},
 '_creation_time': 1764418165659,
 '_last_update_time': 1764418165659}

In [26]:
latest_run_dict = latest_run.to_dictionary()
yaml_str = yaml.dump(latest_run_dict, sort_keys=False, default_flow_style=False)
# Strong bright colors for dark background
formatter = Terminal256Formatter(style="monokai")  
print(highlight(yaml_str, YamlLexer(), formatter))

info:
  artifact_uri: mlflow-artifacts:/1/c2a0423253964f48b4b5adc042eb5a4f/artifacts
  end_time: 1764510510772
  experiment_id: '1'
  lifecycle_stage: active
  run_id: c2a0423253964f48b4b5adc042eb5a4f
  run_name: rnn_16
  start_time: 1764510438174
  status: FINISHED
  user_id: srirama
data:
  metrics:
    accuracy: 0.279920369386673
    loss: 1.374122142791748
    val_accuracy: 0.3031262457370758
    val_loss: 1.3692481517791748
    validation_accuracy: 0.3031262457370758
    validation_loss: 1.3692481517791748
  params:
    epochs: '1'
    batch_size: '400'
    validation_split: '0.0'
    shuffle: 'True'
    class_weight: None
    sample_weight: None
    initial_epoch: '0'
    steps_per_epoch: None
    validation_steps: None
    validation_batch_size: None
    validation_freq: '1'
    opt_name: adam
    opt_learning_rate: '0.0010000000474974513'
    opt_weight_decay: None
    opt_clipnorm: None
    opt_global_clipnorm: None
    opt_clipvalue: None
    opt_use_ema: 'False'
    opt_ema_

In [27]:
latest_run.outputs.model_outputs[0].model_id

'm-e6f1deaa6edf4a7585a42f5fe55b1d3e'

In [28]:
from pprint import pprint
latest_metrics = latest_run.data.metrics
pprint(latest_metrics)

{'accuracy': 0.279920369386673,
 'loss': 1.374122142791748,
 'val_accuracy': 0.3031262457370758,
 'val_loss': 1.3692481517791748,
 'validation_accuracy': 0.3031262457370758,
 'validation_loss': 1.3692481517791748}


In [29]:
latest_run_id = latest_run.info.run_id
latest_run_id


'c2a0423253964f48b4b5adc042eb5a4f'

In [ ]:
# model = mlflow.tensorflow.load_model(f"runs:/{latest_run_id}/model") # Chatgpt (not working for me)
# mlflow.keras.load_model('runs:/81a95b558c1741bfb9c6edc0623242f5/model')
# loaded_model = mlflow.tensorflow.load_model("models:/m-5c1e81e024074e0c9cf1a6e5a3c3e4e5")
# loaded_model = mlflow.tensorflow.load_model('mlflow-artifacts:/1/models/m-5c1e81e024074e0c9cf1a6e5a3c3e4e5/artifacts') # Works
# mlflow-artifacts:/1/models/m-e6f1deaa6edf4a7585a42f5fe55b1d3e/artifacts
loaded_model = mlflow.tensorflow.load_model(f'mlflow-artifacts:/{experiment._experiment_id}/models/{latest_run.outputs.model_outputs[0].model_id}/artifacts') # Works

In [11]:
from pathlib import Path
str(Path().resolve())

'/home/srirama/sr_proj/EmotionAnalysis/test'

In [ ]:
import sys
import mlflow
from mlflow.tensorflow import load_model
from mlflow.tracking import MlflowClient

from pathlib import Path
from loguru import logger
from omegaconf import OmegaConf
from argparse import ArgumentParser

BASE_DIR =  Path(__file__).resolve().parents[2]
sys.path.append(str(BASE_DIR))

def get_arguments():
    parser = ArgumentParser(description="Run training pipeline")
    parser.add_argument(
        "--config_path",
        type=str,
        default="params.yaml",
        help="Path to configuration file"
    )
    return parser.parse_args()

def main():
    args = get_arguments()
    config = OmegaConf.load(BASE_DIR / args.config_path)
    TRACKING_URI = config.mlflow.tracking_uri
    EXPERIMENT_NAME = config.mlflow.experiment_name
    mlflow.set_tracking_uri(TRACKING_URI)
    
    client = MlflowClient(TRACKING_URI)
    experiment = client.get_experiment_by_name(EXPERIMENT_NAME)
    if not experiment:
        raise ValueError(f"Experiment '{EXPERIMENT_NAME}' not found.")

    # Fetch latest FINISHED run
    runs = client.search_runs(
        experiment_ids=[experiment.experiment_id],
        filter_string="attributes.status = 'FINISHED'",
        order_by=["attributes.start_time DESC"],
        max_results=1,
    )
    if not runs:
        raise ValueError("No finished runs found for this experiment.")

    latest_run = runs[0]
    latest_run_id = latest_run.info.run_id
    run_name = latest_run.info.run_name
    
    # Print key metrics prominently
    logger.success(f"Latest Run Name : {run_name} | Run ID : {latest_run_id}")
    metrics = latest_run.data.metrics
    logger.debug(f"Metrics : {metrics}")
    
    model_uri = Path(experiment._artifact_location) / "models" / str(latest_run.outputs.model_outputs[0].model_id) / "artifacts"
    loaded_model = mlflow.tensorflow.load_model(str(model_uri)) # Works
    logger.info(f"Model Loaded Successfully from : {model_uri}")

if __name__ == "__main__":
    main()